In [6]:
! pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 42.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 39.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 48.4 MB/s eta 0:00:00


In [8]:
import pandas as pd
import glob as gb

In [13]:
DIR = 'airflow/logs/dag_id=app_template/*/*/*'
logs_paths = gb.glob(DIR)

In [64]:
df_logs = pd.DataFrame()
for path in logs_paths:
    task_type = path.split('/')[4].split('=')[1].split('.')[0]
    df = pd.read_csv(path, sep=' ', skiprows=6, header=None)
    df = df[df[1]=='{helpers.py:110}']
    col_names = {
        0: 'timestamp',
        4: 'tasks',
        5: 'batch_id',
        6: 'processed_%',
        7: 'seconds'
    }
    df = df[col_names.keys()]
    df.rename(columns=col_names, inplace=True)
    df['task_type'] = task_type
    df_logs = pd.concat([df_logs, df], ignore_index=True)
df_logs.reset_index(drop=True, inplace=True)
df_logs.sample(10)

,timestamp,tasks,batch_id,processed_%,seconds,task_type
308,[2025-06-07T15:17:41.310+0000],1-1-2,60,48.0,1.856,extractors
455,[2025-06-07T15:32:07.771+0000],1-1-4,20,64.52,2.488,loaders
903,[2025-06-07T15:09:28.561+0000],1-1-1,31,24.8,3.925,transformers
269,[2025-06-07T15:16:17.175+0000],1-1-2,21,16.8,2.112,extractors
302,[2025-06-07T15:17:29.420+0000],1-1-2,54,43.2,1.796,extractors
803,[2025-06-07T15:09:30.002+0000],1-1-1,56,44.8,2.259,loaders
349,[2025-06-07T15:19:02.489+0000],1-1-2,101,80.8,1.543,extractors
1433,[2025-06-07T15:25:44.694+0000],1-1-3,63,50.4,2.08,extractors
936,[2025-06-07T15:11:31.828+0000],1-1-1,64,51.2,3.795,transformers
1411,[2025-06-07T15:25:00.525+0000],1-1-3,41,32.8,2.704,extractors
